# Agenda, week 2

1. Q&A
2. dtypes
3. `NaN` (not a number)
4. data frames (2D data structures)
5. Adding and removing data in our data frames
6. Useful methods and attributes
7. Querying with boolean indexes
8. Querying with `.loc`
9. Read some CSV data from a file

# dtypes



In [3]:
import numpy as np   # this is not strictly necessary, but very useful
import pandas as pd  # this is necessary!

from pandas import Series, DataFrame   # this is convenient

In [4]:
# let's create a series

s = Series([10, 20, 30, 40, 50])

s

0    10
1    20
2    30
3    40
4    50
dtype: int64

# What's a dtype?

Many people, when they're learning Python, wonder why we talk about "lists" rather than "arrays." After all, aren't they the same?

No: Lists are different from arrays in two different ways:

- We can change their size (adding and removing items)
- Each object in a list can be of a different type. In an array, they must all be of the same type.

Fast forward to now, when we're working with NumPy and Pandas, and we're really dealing with arrays. That means we cannot change their size (although Pandas does allow for that, thanks to some magic) and all of the elements have to be of the same type.

In the worlds of NumPy and Pandas, that type is known as the "dtype," the data type.

What options do we have for dtypes? These are (mostly) set by NumPy.

Dtypes

- Integers
    - `np.int8`
    - `np.int16`
    - `np.int32`
    - `np.int64` -- the default!
- Unsigned integers
    - `np.uint8`
    - `np.uint16`
    - `np.uint32`
    - `np.uint64`
- Floats
    - `np.float16`
    - `np.float32`
    - `np.float64` -- the default!
    - `np.float128`
    
# What does this mean?

If you don't specify a dtype when you create a series, Pandas will guess what you want/need:

- If you have only integers, then it'll use `np.int64`
- If you have any floating-point numbers, then it'll use `np.float64`
- If you have strings or other funny Python objects, then it'll use `object` as its type

In [5]:
# we can get the dtype of a series by retrieving the dtype attribute

s.dtype

dtype('int64')

If you don't want to specify `np.int8`, then you can instead say `'int8'`, and it'll work the same way.

You can also say `np.dtype('int64')`.

In [8]:
s = Series([10, 20, 30, 40, 50])
s

0    10
1    20
2    30
3    40
4    50
dtype: int64

In [9]:
# how can we specify a different dtype?
# when we create a series, we can pass the keyword argument dtype= along with a valid dtype.

s = Series([10, 20, 30, 40, 50], dtype=np.int8)   # 8-bit numbers
s

0    10
1    20
2    30
3    40
4    50
dtype: int8

In [10]:
# let's multiply our series (s) by 100!

# I can use broadcasting

s * 100

0    -24
1    -48
2    -72
3    -96
4   -120
dtype: int8

In [11]:
# what happened? 8 bits (signed) aren't enough to hold 1,000 let alone larger numbers.
# so, sort of like a car odometer or an old-style videogame, the numbers roll over
# this is very very bad -- you won't get a warning!



# This is why you need to worry

If your dtype is too small, then if the numbers get too big, you'll lose data without any warning.

So, why not just use larger dtypes? Because that can be a waste of memory.

Imagine 1m 64-bit ints. That'll take up ... 64 MB.

Imagine 1m 8-bit ints. That'll take up 8 MB.

That might not seem like a lot nowadays.  But what if we have 1b rows?

Then it's the difference between 64 GB and 8 GB.. and that's already serious.

So you have to balance between a dtype that's not too small (and won't cause data loss) and not too big (and won't overwhelm your system).  This isn't always easy!

In [12]:
s1 = Series([10, 20, 30, 40, 50])
s2 = Series([90, 91, 92.3, 94, 95])


In [13]:
s1

0    10
1    20
2    30
3    40
4    50
dtype: int64

In [14]:
s2

0    90.0
1    91.0
2    92.3
3    94.0
4    95.0
dtype: float64

In [15]:
s1 + s2    # each of the operations will be int + float, which gives us back a float

0    100.0
1    111.0
2    122.3
3    134.0
4    145.0
dtype: float64

In [16]:
# how can I change the dtype of a series?
# what does that even mean?

# if I change the dtype from int to float, we won't lose any data
# if I change the dtype from float to int, I might well lose data... what happens?

# You cannot change the dtype of a series
s.dtype = np.float64

AttributeError: property 'dtype' of 'Series' object has no setter

In [17]:
# we can create a new series, based on our existing series, with a different dtype
# if we do this, by calling the "astype" method, the new series will have the new dtype
# and each element will go through the appropriate transformation

# floats turned to ints will be truncated, for example

s

0    10
1    20
2    30
3    40
4    50
dtype: int8

In [18]:
s.astype(np.float64)   # new series, based on s

0    10.0
1    20.0
2    30.0
3    40.0
4    50.0
dtype: float64

In [19]:
# what about textual data?

s = Series('hello out there to everyone'.split())
s

0       hello
1         out
2       there
3          to
4    everyone
dtype: object

In [20]:
# we'll talk more about text strings in week 4. You should know that text strings have a dtype
# of "object", because they're using regular Python strings, and referring to them there.

# Exercise: Mean from strings

1. Create a list of strings, in which each string contains only digits
2. Create a series based on that list.
3. Transform the series such that you can calculate the mean of those numbers.

Example:

If my list is `[10, 20, 30]`, then I want to have series such that I can call `s.mean()` and get back 20.    

In [21]:
mylist = '11 15 23 97 65'.split()
mylist

['11', '15', '23', '97', '65']

In [22]:
s = Series(mylist)

In [23]:
s

0    11
1    15
2    23
3    97
4    65
dtype: object

In [24]:
# what happens when I try to calculate the mean on them?
s.mean()

223047953.0

In [25]:
# basically, Pandas added together all of the *strings*
s.sum()

'1115239765'

In [26]:
int(s.sum()) / 5

223047953.0

In [28]:
# if we really want to get the mean of these numbers,
# we'll need to transform our series into one of integers

s.astype(np.int8).mean()

42.2

In [29]:
# another way to do this would be at series creation time

s = Series(mylist, dtype=np.int8)

/usr/local/lib/python3.11/site-packages/numpy/core/numeric.py:2468: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  return bool(asarray(a1 == a2).all())


In [30]:
s

0    11
1    15
2    23
3    97
4    65
dtype: int8